In [125]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import copy
import numpy as np
import pandas as pd
import timeit
from IPython.display import display

from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image

from layers import Flatten

In [4]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

In [5]:
# Load data
train_path = './input/train-jpg/'
test_path = './input/test-jpg/'
train = pd.read_csv('./input/train_v2.csv')
test = pd.read_csv('./input/sample_submission_v2.csv')

In [6]:
print(train.shape)
print(test.shape)
NUM_TRAIN = 32000
NUM_VAL = train.shape[0]-NUM_TRAIN
NUM_TEST = test.shape[0]

(40479, 2)
(61191, 2)


In [7]:
print(train.iloc[0])
print(test.iloc[0])

image_name         train_0
tags          haze primary
Name: 0, dtype: object
image_name                                  test_0
tags          primary clear agriculture road water
Name: 0, dtype: object


In [8]:
display(train[0:10])

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [9]:
train['tags'][0].split()

['haze', 'primary']

TBD
Make loop to import all images & store as a numpy array of (3,32,32)'s
Save extracted image data somewhere so I don't need to preprocess each time
Convert text labels into multi-hot vectors, with vocab as the 17 labels in alphabetical order. 1 = agriculture, 2 = clear, etc.

In [22]:
vocab = [item for i in range(train.shape[0]) for item in train['tags'][i].split()]
vocab_ordered = sorted(set(vocab))
vocab_dict = {word: index for index, word in enumerate(vocab_ordered)}
vocab_dict

{'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}

In [28]:
labels_inds = [[vocab_dict[word] for word in row.split()] for row in train['tags']] 

In [58]:
mlb = MultiLabelBinarizer()
labels_words = [set([word for word in row.split()]) for row in train['tags']]
labels = mlb.fit_transform(labels_words)

In [87]:
train_dataset = np.zeros((100,3,256,256))
for i,image_name in enumerate(train['image_name'][:100]):
    im = Image.open(train_path + image_name + '.jpg')
    im = np.array(im)[:,:,:3]
    im = np.reshape(im,(im.shape[2],im.shape[0],im.shape[1]))
    train_dataset[i,:,:,:] = im

In [88]:
train_dataset.shape

(100, 3, 256, 256)

In [120]:
train_data = torch.from_numpy(train_dataset)
our_labels = torch.from_numpy(labels[:100])
train_tensor_dataset = torch.utils.data.TensorDataset(train_data, our_labels)
print(train_tensor_dataset[0])
loader_train = torch.utils.data.DataLoader(train_tensor_dataset, batch_size=10, shuffle=True)

(
( 0 ,.,.) = 
  158  143  150  ...   146  153  163
  146  153  160  ...   151  164  148
  151  165  148  ...   160  146  152
      ...         ⋱        ...      
  147  156  166  ...   149  160  146
  152  161  145  ...   160  141  150
  166  147  153  ...   149  156  171

( 1 ,.,.) = 
  148  157  168  ...   147  159  145
  150  160  144  ...   161  141  149
  166  147  153  ...   149  154  169
      ...         ⋱        ...      
  148  159  142  ...   167  143  152
  160  144  158  ...   146  150  162
  146  151  160  ...   145  157  139

( 2 ,.,.) = 
  146  158  140  ...   166  146  151
  159  144  158  ...   146  150  164
  146  151  164  ...   143  156  137
      ...         ⋱        ...      
  157  147  149  ...   140  147  153
  140  145  153  ...   156  162  151
  157  162  149  ...   172  153  157
[torch.DoubleTensor of size 3x256x256]
, 
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
[torch.LongTensor of size 17]
)


In [133]:
simple_model = nn.Sequential(
                nn.Conv2d(3, 3, kernel_size=3, stride=1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(3),
                nn.MaxPool2d(kernel_size=2,stride=2),
                Flatten(),
                nn.Linear(48387,17)
              )
gpu_dtype = torch.cuda.FloatTensor
simple_model.type(gpu_dtype)

#check output dimensions before flattening
model_gpu = copy.deepcopy(simple_model).type(gpu_dtype)
model_gpu.eval()
x = torch.randn(10, 3, 256, 256).type(gpu_dtype)
x_var = Variable(x.type(gpu_dtype)) # Construct a PyTorch Variable out of your input data
scores = model_gpu(x_var)        # Feed it through the model! 
print(scores.size())

loss_fn = nn.MultiLabelSoftMarginLoss().type(gpu_dtype)
optimizer = optim.RMSprop(simple_model.parameters(), lr=1e-3, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

torch.Size([10, 17])


In [135]:
simple_model.train()

print_every = 5

# Load one batch at a time.
for t, (x, y) in enumerate(loader_train):
    x_var = Variable(x.type(gpu_dtype))
    y_var = Variable(y.type(gpu_dtype))

    # This is the forward pass: predict the scores for each class, for each x in the batch.
    scores = simple_model(x_var)
    
    # Use the correct y values and the predicted y values to compute the loss.
    loss = loss_fn(scores, y_var)
    
    if (t + 1) % print_every == 0:
        print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

    # Zero out all of the gradients for the variables which the optimizer will update.
    optimizer.zero_grad()
    
    # This is the backwards pass: compute the gradient of the loss with respect to each 
    # parameter of the model.
    loss.backward()
    
    # Actually update the parameters of the model using the gradients computed by the backwards pass.
    optimizer.step()

t = 5, loss = 6.7969
t = 10, loss = 3.0138
